# 1. Import libraries

In [1]:
#----------------------------Reproducible----------------------------------------------------------------------------------------
import numpy as np
import random as rn
import os

seed=0
os.environ['PYTHONHASHSEED'] = str(seed)

np.random.seed(seed)
rn.seed(seed)

#----------------------------Reproducible----------------------------------------------------------------------------------------

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

#--------------------------------------------------------------------------------------------------------------------------------
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
matplotlib.style.use('ggplot')

import random
import scipy.sparse as sparse
import scipy.io

from keras.utils import to_categorical
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from skfeature.utility import construct_W
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skfeature.function.sparse_learning_based import MCFS
from skfeature.utility import construct_W
from skfeature.utility.sparse_learning import feature_ranking
import time
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer 
import pandas as pd
from skimage import io
from PIL import Image
from sklearn.model_selection import train_test_split
import scipy.sparse as sparse
import scipy.io
#--------------------------------------------------------------------------------------------------------------------------------
#Import ourslef defined methods
import sys
sys.path.append(r"../Defined")
import Functions as F

Using TensorFlow backend.


# 2. Loading data

In [2]:
data_path="../Dataset/Prostate_GE.mat"
Data = scipy.io.loadmat(data_path)

data_arr_=Data['X']
label_arr=Data['Y'][:, 0]

Data=MinMaxScaler(feature_range=(0,1)).fit_transform(data_arr_)

data_arr=Data
label_arr_onehot=label_arr

In [3]:
key_feture_number=64

In [4]:
num_cluster=len(np.unique(label_arr_onehot))

# 3 .Calculation

In [5]:
#--------------------------------------------------------------------------------------------------------------------------------
def write_to_csv(p_data,p_path):
    dataframe = pd.DataFrame(p_data)
    dataframe.to_csv(p_path, mode='a',header=False,index=False,sep=',')
    del dataframe

#--------------------------------------------------------------------------------------------------------------------------------       
def mse_check(train, test):
    LR = LinearRegression(n_jobs = -1)
    LR.fit(train[0], train[1])
    MSELR = ((LR.predict(test[0]) - test[1]) ** 2).mean()
    return MSELR
 
#--------------------------------------------------------------------------------------------------------------------------------       
def cal(p_data_arr,\
        p_label_arr_onehot,\
        p_key_feture_number,\
        p_seed):
    
    C_train_x,C_test_x,C_train_y,C_test_y= train_test_split(p_data_arr,p_label_arr_onehot,test_size=0.2,random_state=p_seed)

    os.environ['PYTHONHASHSEED'] = str(p_seed)
    np.random.seed(p_seed)
    rn.seed(p_seed)
    
    train_feature=C_train_x
    train_label=C_train_y
    test_feature=C_test_x
    test_label=C_test_y
    
    #--------------------------------------------------------------------------------------------------------------------------------
    # construct affinity matrix
    kwargs_W =  {"metric": "euclidean", "neighborMode": "knn", "weightMode": "heatKernel", "k": 5, 't': 1}
    
    t_start = time.time()

    train_W = construct_W.construct_W(train_feature, **kwargs_W)

    # Obtain the scores of features, and sort the feature scores in an ascending order according to the feature scores
    tain_score = MCFS.mcfs(train_feature, n_selected_features=key_feture_number, W=train_W,n_clusters=num_cluster)
    train_idx = MCFS.feature_ranking(tain_score)
    
    t_used=time.time() - t_start
    write_to_csv(np.array([t_used]),"./log/MCFS_time.csv")

    C_train_selected_x = train_feature[:, train_idx[0:key_feture_number]]

    test_W = construct_W.construct_W(test_feature, **kwargs_W)

    # Obtain the scores of features, and sort the feature scores in an ascending order according to the feature scores
    test_score = MCFS.mcfs(test_feature, n_selected_features=key_feture_number, W=test_W,n_clusters=num_cluster)
    test_idx = MCFS.feature_ranking(test_score)

    # Obtain the dataset on the selected features
    C_test_selected_x = test_feature[:, test_idx[0:key_feture_number]]

    # Classification on original features
    train_feature=C_train_x
    train_label=C_train_y
    test_feature=C_test_x
    test_label=C_test_y
    
    orig_train_acc,orig_test_acc=F.ETree(train_feature,train_label,test_feature,test_label,0)
    
    # Classification on selected features
    
    train_feature=C_train_selected_x
    train_label=C_train_y
    test_feature=C_test_selected_x
    test_label=C_test_y

    selec_train_acc,selec_test_acc=F.ETree(train_feature,train_label,test_feature,test_label,0)

    # Linear reconstruction
    train_feature_tuple=(C_train_selected_x,C_train_x)
    test_feature_tuple=(C_test_selected_x,C_test_x)

    reconstruction_loss=mse_check(train_feature_tuple, test_feature_tuple)
    results=np.array([orig_train_acc,orig_test_acc,selec_train_acc,selec_test_acc,reconstruction_loss])
    
    write_to_csv(results.reshape(1,len(results)),"./log/MCFS_results.csv")
    
    return orig_train_acc,orig_test_acc,selec_train_acc,selec_test_acc,reconstruction_loss

In [6]:
p_data_arr=data_arr
p_label_arr_onehot=label_arr_onehot
p_key_feture_number=key_feture_number

In [ ]:
for p_seed in np.arange(0,50):
    orig_train_acc,orig_test_acc,selec_train_acc,selec_test_acc,reconstruction_loss=cal(p_data_arr,\
                                                                                        p_label_arr_onehot,\
                                                                                        p_key_feture_number,\
                                                                                        p_seed)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 26 iterations, i.e. alpha=6.071e-03, with an active set of 20 regressors, and the smallest cholesky pivot element being 6.234e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 28 iterations, i.e. alpha=6.059e-03, with an active set of 21 regressors, and the smallest cholesky pivot element being 1.490e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 28 iterations, i.e. alpha=6.059e-03, with an active set of 21 regressors, and the smallest cholesky pivot el

Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.9523809523809523
Testing accuracy： 0.9523809523809523
Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.5238095238095238
Testing accuracy： 0.5238095238095238


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 26 iterations, i.e. alpha=7.971e-03, with an active set of 21 regressors, and the smallest cholesky pivot element being 7.146e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)


Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.8095238095238095
Testing accuracy： 0.8095238095238095
Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.6190476190476191
Testing accuracy： 0.6190476190476191


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 28 iterations, i.e. alpha=5.522e-03, with an active set of 20 regressors, and the smallest cholesky pivot element being 6.989e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 28 iterations, i.e. alpha=5.517e-03, with an active set of 20 regressors, and the smallest cholesky pivot element being 3.942e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 28 iterations, i.e. alpha=5.447e-03, with an active set of 20 regressors, and the smallest cholesky pivot el

...

Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.8571428571428571
Testing accuracy： 0.8571428571428571
Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.2857142857142857
Testing accuracy： 0.2857142857142857


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 23 iterations, i.e. alpha=1.004e-03, with an active set of 20 regressors, and the smallest cholesky pivot element being 2.980e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 23 iterations, i.e. alpha=1.002e-03, with an active set of 20 regressors, and the smallest cholesky pivot element being 4.470e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 23 iterations, i.e. alpha=1.002e-03, with an active set of 20 regressors, and the smallest cholesky pivot el

Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.9523809523809523
Testing accuracy： 0.9523809523809523
Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.42857142857142855
Testing accuracy： 0.42857142857142855


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 30 iterations, i.e. alpha=9.038e-03, with an active set of 21 regressors, and the smallest cholesky pivot element being 4.942e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 30 iterations, i.e. alpha=8.809e-03, with an active set of 21 regressors, and the smallest cholesky pivot element being 8.560e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 30 iterations, i.e. alpha=8.780e-03, with an active set of 21 regressors, and the smallest cholesky pivot el

Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.9047619047619048
Testing accuracy： 0.9047619047619048
Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.42857142857142855
Testing accuracy： 0.42857142857142855
